# Extending

You can extend FD-Shifts to benchmark your own models including new confidence scores, your own dataset and your own softmax-based confidence scoring functions. In this tutorial we will see how.

## Some Setup

First we have to import a lot of stuff and create a config object. Make sure to set `EXPERIMENT_ROOT_DIR` and `DATASET_ROOT_DIR` appropriately beforehand.

### Setup on local machine
The following steps are necessary to run this notebook with all features in your local jupyter notebook environment.
1. Open Terminal
2. `$ conda activate {fd-shifts-environment}`
2. `$ pip install ipykernel`
3. `$ python -m ipykernel install --name fd-shifts --user`
4. `$ pip install ipywidgets`

Now the kernel `fd-shifts` can be selected in jupyter notebooks alongside visualizations.

In [7]:
import os

cwd = os.getcwd()
fd_dir = os.path.abspath(os.path.join(cwd, os.pardir))
os.environ["EXPERIMENT_ROOT_DIR"] = fd_dir + "/experiments_folder"
os.environ["DATASET_ROOT_DIR"] = fd_dir + "/data_folder"

In [2]:
from typing import Callable, Optional
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks.progress.rich_progress import RichProgressBar
from rich import print
from torch import nn
from torchvision import datasets, transforms

from fd_shifts import analysis, configs, models, reporting
from fd_shifts.exec import test, train
from fd_shifts.loaders import dataset_collection
from fd_shifts.loaders.abstract_loader import AbstractDataLoader
from fd_shifts.utils import exp_utils

configs.init()
config = configs.Config.with_defaults(data="svhn")

# Remove tinyimagenet, you can comment this out if you downloaded tinyimagenet_resize and put it
# into $DATASET_ROOT_DIR
config.eval.query_studies.new_class_study = ["cifar10", "cifar100"]

/home/c817h/anaconda3/envs/fd-shifts/lib/python3.10/site-packages/fd_shifts/exec.py:165: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="configs", config_name="config")


## Adding a New Model

Let's start with adding a new model. First we set up a model class inheriting from `LightningModule`. We have to create methods for `train`, `validation` and `test` steps for the benchmark to work. We will also copy over `load_only_state_dict`, a helper method.

In [3]:
class MyModel(pl.LightningModule):
    def __init__(self, cfg: configs.Config) -> None:
        super().__init__()

        self.cfg = cfg
        self.ext_confid_name = "my_fancy_confidence"

        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(
                cfg.data.img_size[0] * cfg.data.img_size[1] * cfg.data.img_size[2], 512
            ),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
        )

        self.classifier = nn.Linear(512, 10)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.classifier(self.encoder(x))
        loss = torch.nn.functional.cross_entropy(logits, y)
        return {"loss": loss, "softmax": torch.softmax(logits, dim=1), "labels": y}

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch

        if x.shape[1] > 1 and self.cfg.data.img_size[2] == 1:
            x = transforms.Grayscale()(x)

        logits = self.classifier(self.encoder(x))
        loss = torch.nn.functional.cross_entropy(logits, y)
        my_fancy_confidence = torch.sum(logits, dim=1)
        return {
            "loss": loss,
            "softmax": torch.softmax(logits, dim=1),
            "labels": y,
            "confid": my_fancy_confidence,
        }

    def test_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch

        if x.shape[1] > 1 and self.cfg.data.img_size[2] == 1:
            x = transforms.Grayscale()(x)

        logits = self.classifier(self.encoder(x))
        my_fancy_confidence = torch.sum(logits, dim=1)
        self.test_results = {
            "logits": logits,
            "labels": y,
            "confid": my_fancy_confidence,
        }

    def load_only_state_dict(self, path):
        ckpt = torch.load(path)
        self.load_state_dict(ckpt["state_dict"], strict=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        return {
            "optimizer": optimizer,
            "lr_scheduler": torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer, T_max=self.cfg.trainer.num_epochs
            ),
        }

To use the model we have to tell FD-Shifts about it and update our configuration. We will also use this opportunity to update the default configuration with some information about our experiment. Afterwards we can train the model. In this case we train on SVHN since that is the dataset for which we loaded the default configuration.

In [4]:
models.register_model("my_model", MyModel)

config.model.name = "my_model"
config.eval.ext_confid_name = "ext"

config.trainer.num_epochs = 10
config.trainer.batch_size = 256

group_name = config.data.dataset
name = "my_first_experiment"
group_dir = config.exp.group_dir.parent / group_name
exp_dir = group_dir / name
exp_dir.mkdir(exist_ok=True, parents=True)
version = exp_utils.get_next_version(exp_dir)

config.exp = configs.ExperimentConfig(
    group_name=group_name,
    name=name,
    mode=configs.Mode.train,
    fold=0,
    crossval_n_folds=0,
    global_seed=1234,
    version=version,
    work_dir=os.getcwd(),
    data_root_dir=os.getenv("DATASET_ROOT_DIR"),
    group_dir=group_dir,
    dir=exp_dir,
    version_dir=exp_dir / f"version_{version}",
    output_paths=configs.OutputPathsPerMode(
        test=configs.OutputPathsConfig(
            raw_output=exp_dir / "test_results" / "raw_logits.npz",
            raw_output_dist=exp_dir / "test_results" / "raw_logits_dist.npz",
            external_confids=exp_dir / "test_results" / "external_confids.npz",
            external_confids_dist=exp_dir
            / "test_results"
            / "external_confids_dist.npz",
        )
    ),
)

print(config)

Config(
    pkgversion='0.1.dev420+g24be4a2.d20221201',
    data=DataConfig(
        dataset='svhn',
        data_dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/data/svhn'),
        pin_memory=True,
        img_size=(32, 32, 3),
        num_workers=12,
        num_classes=10,
        reproduce_confidnet_splits=True,
        augmentations={
            'train': {
                'to_tensor': None,
                'normalize': [[0.4376821, 0.4437697, 0.47280442], [0.19803012, 0.20101562, 0.19703614]]
            },
            'val': {
                'to_tensor': None,
                'normalize': [[0.4376821, 0.4437697, 0.47280442], [0.19803012, 0.20101562, 0.19703614]]
            },
            'test': {
                'to_tensor': None,
                'normalize': [[0.4376821, 0.4437697, 0.47280442], [0.19803012, 0.20101562, 0.19703614]]
            }
        },
        kwargs=None
    ),
    trainer=TrainerConfig(
        accumulate_grad_batches=1,
        resume_from_ckpt_confidnet=False,
        num_epochs=10,
        num_steps=None,
        num_epochs_backbone=100,
        dg_pretrain_epochs=50,
        val_every_n_epoch=5,
        val_split=<ValSplit.devries: 'devries'>,
        do_val=True,
        batch_size=256,
        resume_from_ckpt=False,
        benchmark=True,
        fast_dev_run=False,
        lr_scheduler=LRSchedulerConfig(_target_='torch.optim.lr_scheduler.CosineAnnealingLR', _partial_=True),
        optimizer=OptimizerConfig(_target_='torch.optim.sgd.SGD', _partial_=True),
        callbacks={'model_checkpoint': None, 'confid_monitor': None, 'learning_rate_monitor': None},
        learning_rate_confidnet=None,
        learning_rate_confidnet_finetune=None
    ),
    exp=ExperimentConfig(
        group_name='svhn',
        name='my_first_experiment',
        version=2,
        mode=<Mode.train: 'train'>,
        work_dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs'),
        fold_dir=PosixPath('???'),
        root_dir=PosixPath('???'),
        data_root_dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/data'),
        group_dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/svh
n'),
        dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/svhn/my_f
irst_experiment'),
        version_dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/s
vhn/my_first_experiment/version_2'),
        fold=0,
        crossval_n_folds=0,
        crossval_ids_path=PosixPath('???'),
        output_paths=OutputPathsPerMode(
            fit=OutputPathsConfig(
                input_imgs_plot=None,
                raw_output=PosixPath('???'),
                raw_output_dist=PosixPath('???'),
                external_confids=PosixPath('???'),
                external_confids_dist=PosixPath('???')
            ),
            test=OutputPathsConfig(
                input_imgs_plot=None,
                raw_output=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experi
ments/svhn/my_first_experiment/test_results/raw_logits.npz'),
                raw_output_dist=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/e
xperiments/svhn/my_first_experiment/test_results/raw_logits_dist.npz'),
                external_confids=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/
experiments/svhn/my_first_experiment/test_results/external_confids.npz'),
                external_confids_dist=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/
docs/experiments/svhn/my_first_experiment/test_results/external_confids_dist.npz')
            )
        ),
        log_path=PosixPath('???'),
        global_seed=1234
    ),
    model=ModelConfig(
        name='my_model',
        fc_dim=512,
  

In [5]:
train(config, RichProgressBar())

2022-12-01 14:11:13.385 | INFO     | fd_shifts.exec:train:31 - CHECK CUDNN VERSION
2022-12-01 14:11:13.386 | WARNING  | fd_shifts.utils.exp_utils:set_seed:14 - SETTING GLOBAL SEED
Global seed set to 1234
2022-12-01 14:11:13.389 | INFO     | fd_shifts.exec:train:37 - setting seed 1234, benchmark to False for deterministic training.
2022-12-01 14:11:13.424 | INFO     | fd_shifts.models.callbacks:get_callbacks:34 - Adding ModelCheckpoint callback
/home/c817h/anaconda3/envs/fd-shifts/lib/python3.10/site-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py:52: LightningDeprecationWarning: Setting `max_steps = None` is deprecated in v1.5 and will no longer be supported in v1.7. Use `max_steps = -1` instead.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
2022-12-01 14:11:13.505 

Using downloaded and verified file: /home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/data/svhn/train_32x32.mat
Using downloaded and verified file: /home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/data/svhn/test_32x32.mat
Using downloaded and verified file: /home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/data/svhn/test_32x32.mat
Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name       ┃ Type       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ encoder    │ Sequential │  1.8 M │
│ 1 │ classifier │ Linear     │  5.1 K │
└───┴────────────┴────────────┴────────┘

Trainable params: 1.8 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.8 M                                                                                                
Total estimated model params size (MB): 7

...

dict_keys(['det_mcp']) [1000]

CHECK VAL METRICS {'loss': 0.6642002463340759, 'nll': 0.662044882774353, 'accuracy': 0.8358027935028076, 
'brier_score': 0.2508915662765503, 'det_mcp_failauc': 0.8652118100128369, 'det_mcp_fpr@95tpr': 0.6341463414634146, 
'det_mcp_failap_suc': 0.9700720634580682, 'det_mcp_failap_err': 0.5030212019120829, 'det_mcp_aurc': 
40.344537490264116, 'det_mcp_e-aurc': 26.09443018051993}

CHECK TRAIN METRICS {'loss': 0.23697537183761597, 'nll': 0.2367718517780304, 'accuracy': 0.9357409477233887, 
'det_mcp_failauc': 0.94176432216379, 'det_mcp_fpr@95tpr': 0.32972858354537743, 'det_mcp_failap_suc': 
0.9954858274174538, 'det_mcp_failap_err': 0.5250951160683579, 'det_mcp_aurc': 6.391118522732987, 'det_mcp_e-aurc': 
4.273022877176052}

saved raw validation outputs to ???

saved external confids validation outputs to ???

Now let's set up some information about the testing and test on SVHN as well as some additional datasets.

In [6]:
config.test = configs.TestConfig(
    name="test_results",
    dir=exp_dir / "test_results",
    cf_path=exp_dir / "hydra/config.yaml",
    selection_criterion="latest",
    best_ckpt_path=exp_dir / f"version_{version}/latest.ckpt",
    only_latest_version=True,
    devries_repro_ood_split=False,
    assim_ood_norm_flag=False,
    iid_set_split="devries",
    raw_output_path="raw_logits.npz",
    external_confids_output_path="external_confids.npz",
    selection_mode="max",
    output_precision=64,
)


print(config)

test(config)

Config(
    pkgversion='0.1.dev420+g24be4a2.d20221201',
    data=DataConfig(
        dataset='svhn',
        data_dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/data/svhn'),
        pin_memory=True,
        img_size=(32, 32, 3),
        num_workers=12,
        num_classes=10,
        reproduce_confidnet_splits=True,
        augmentations={
            'train': {
                'to_tensor': None,
                'normalize': [[0.4376821, 0.4437697, 0.47280442], [0.19803012, 0.20101562, 0.19703614]]
            },
            'val': {
                'to_tensor': None,
                'normalize': [[0.4376821, 0.4437697, 0.47280442], [0.19803012, 0.20101562, 0.19703614]]
            },
            'test': {
                'to_tensor': None,
                'normalize': [[0.4376821, 0.4437697, 0.47280442], [0.19803012, 0.20101562, 0.19703614]]
            },
            'external_cifar10': {'to_tensor': None, 'normalize': [[0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 
0.201]]},
            'external_cifar100': {'to_tensor': None, 'normalize': [[0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 
0.201]]}
        },
        kwargs=None
    ),
    trainer=TrainerConfig(
        accumulate_grad_batches=1,
        resume_from_ckpt_confidnet=False,
        num_epochs=10,
        num_steps=None,
        num_epochs_backbone=100,
        dg_pretrain_epochs=50,
        val_every_n_epoch=5,
        val_split=<ValSplit.devries: 'devries'>,
        do_val=True,
        batch_size=256,
        resume_from_ckpt=False,
        benchmark=False,
        fast_dev_run=False,
        lr_scheduler=LRSchedulerConfig(_target_='torch.optim.lr_scheduler.CosineAnnealingLR', _partial_=True),
        optimizer=OptimizerConfig(_target_='torch.optim.sgd.SGD', _partial_=True),
        callbacks={'model_checkpoint': None, 'confid_monitor': None, 'learning_rate_monitor': None},
        learning_rate_confidnet=None,
        learning_rate_confidnet_finetune=None
    ),
    exp=ExperimentConfig(
        group_name='svhn',
        name='my_first_experiment',
        version=1,
        mode=<Mode.train: 'train'>,
        work_dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs'),
        fold_dir=PosixPath('???'),
        root_dir=PosixPath('???'),
        data_root_dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/data'),
        group_dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/svh
n'),
        dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/svhn/my_f
irst_experiment'),
        version_dir=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/s
vhn/my_first_experiment/version_1'),
        fold=0,
        crossval_n_folds=0,
        crossval_ids_path=PosixPath('???'),
        output_paths=OutputPathsPerMode(
            fit=OutputPathsConfig(
                input_imgs_plot=None,
                raw_output=PosixPath('???'),
                raw_output_dist=PosixPath('???'),
                external_confids=PosixPath('???'),
                external_confids_dist=PosixPath('???')
            ),
            test=OutputPathsConfig(
                input_imgs_plot=None,
                raw_output=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experi
ments/svhn/my_first_experiment/test_results/raw_logits.npz'),
                raw_output_dist=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/e
xperiments/svhn/my_first_experiment/test_results/raw_logits_dist.npz'),
                external_confids=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/
experiments/svhn/my_first_experiment/test_results/external_confids.npz'),
                external_confids_dist=PosixPath('/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/
doc

2022-12-01 14:10:07.088 | INFO     | fd_shifts.exec:test:124 - CHECK cf.exp.dir
2022-12-01 14:10:07.089 | DEBUG    | fd_shifts.utils.exp_utils:get_most_recent_version:39 - [1, 0]
2022-12-01 14:10:07.089 | INFO     | fd_shifts.exec:test:128 - testing model from checkpoint: /home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/svhn/my_first_experiment/version_1/last.ckpt from model selection tpye latest
2022-12-01 14:10:07.090 | INFO     | fd_shifts.exec:test:133 - logging testing to: /home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/svhn/my_first_experiment/test_results
2022-12-01 14:10:07.120 | INFO     | fd_shifts.models.callbacks:get_callbacks:34 - Adding ModelCheckpoint callback
/home/c817h/anaconda3/envs/fd-shifts/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:92: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_

Using downloaded and verified file: /home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/data/svhn/train_32x32.mat
Using downloaded and verified file: /home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/data/svhn/test_32x32.mat
Using downloaded and verified file: /home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/data/svhn/test_32x32.mat
Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/c817h/anaconda3/envs/fd-shifts/lib/python3.10/site-packages/rich/live.py:229: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

saved raw test outputs to 
/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/svhn/my_first_experiment/test_re
sults/raw_logits.npz

saved ext confid raw test outputs to 
/home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/svhn/my_first_experiment/test_re
sults/external_confids.npz

2022-12-01 14:10:13.891 | DEBUG    | fd_shifts.analysis:main:929 - starting analysis with in_path /home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/svhn/my_first_experiment/test_results, out_path /home/c817h/Documents/projects/iclr23/failure-detection-benchmark/docs/experiments/svhn/my_first_experiment/test_results, and query studies QueryStudiesConfig(iid_study='svhn', noise_study=[], in_class_study=[], new_class_study=['cifar10', 'cifar100'])
2022-12-01 14:10:13.912 | DEBUG    | fd_shifts.analysis:__init__:322 - CHECK QUERY CONFIDS
['det_mcp', 'det_pe', 'ext', 'det_mls']
2022-12-01 14:10:13.922 | DEBUG    | fd_shifts.analysis:compute_confid_metrics:482 - val_tuning
det_mcp
2022-12-01 14:10:13.928 | DEBUG    | fd_shifts.analysis.eval_utils:get_metrics_per_confid:195 - CHECK FAIL NLL: 
0.9999999999999929
0.20916556671342637
2022-12-01 14:10:13.951 | DEBUG    | fd_shifts.analysis.eval_utils:get_val_risk_scores:329 - STRAIGHT FROM THRESH CALCULATION
0.164

2022-12-01 14:10:16.622 | DEBUG    | fd_shifts.analysis:compute_confid_metrics:482 - new_class_study_cifar10_proposed_mode
det_pe
2022-12-01 14:10:16.898 | DEBUG    | fd_shifts.analysis.eval_utils:get_metrics_per_confid:195 - CHECK FAIL NLL: 
0.9493515969114704
0.08227142634049166
2022-12-01 14:10:16.899 | DEBUG    | fd_shifts.analysis:compute_confid_metrics:590 - checking in
None
det_pe
2022-12-01 14:10:16.899 | DEBUG    | fd_shifts.analysis:compute_confid_metrics:482 - new_class_study_cifar10_proposed_mode
ext
/home/c817h/anaconda3/envs/fd-shifts/lib/python3.10/site-packages/fd_shifts/analysis/metrics.py:235: RuntimeWarning: invalid value encountered in log
  stats_cache.correct * np.log(stats_cache.confids + 1e-7)
/home/c817h/anaconda3/envs/fd-shifts/lib/python3.10/site-packages/fd_shifts/analysis/metrics.py:236: RuntimeWarning: invalid value encountered in log
  + (1 - stats_cache.correct) * np.log(1 - stats_cache.confids + 1e-7)
2022-12-01 14:10:17.173 | DEBUG    | fd_shifts.analy

All computed metrics are now found in various `csv` files in the experiment folder. Let's load and preprocess them and then render a benchmark table similar to the paper.

In [7]:
data = pd.concat(
    [
        reporting.load_file(p, experiment_override=str(p.parent.parent.parent.stem))
        for p in Path(os.getenv("EXPERIMENT_ROOT_DIR")).glob("**/test_results/*.csv")
    ]
)
# data = data.assign(experiment=config.exp.group_name)
data = data.assign(study=data.experiment + "_" + data.study)
data = reporting.assign_hparams_from_names(data)

# data = reporting.filter_unused(data)
data = reporting.rename_confids(data)
data = reporting.rename_studies(data)
data = reporting.tables.aggregate_over_runs(data)
data = reporting.str_format_metrics(data)

results_table = reporting.tables.build_results_table(
    data=data, metric="aurc", original_mode=False, paper_filter=False
)
results_table

svhn            
study          iid ns-ncs     
ncs-data set          c10 c100
MY-EXT CNN     204    332  352
MY-MLS CNN    54.2    161  158
MY-MSR CNN    38.6    132  132
MY-PE  CNN    39.5    133  133

## Adding a New Dataset

We can also evaluate our model (or one of the built in ones) on a custom dataset. Let's define a dataset that is just a wrapper around MNIST for simplicity and tell FD-Shifts about it.

In [ ]:
class MyDataset(datasets.MNIST):
    def __init__(
        self,
        root: str,
        train: bool = True,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        download: bool = False,
    ) -> None:
        super().__init__(root, train, transform, target_transform, download)


dataset_collection.register_dataset("mydataset", MyDataset)

In [ ]:
print(config)

We now have to update our configuration with the new dataset. We will also update the list of datasets to additionally test on. Since we train on MNIST we could argue that SVHN provides a sub-class shift, while CIFAR-10 might be a good choice for a new-class shift.

In [ ]:
from copy import deepcopy
config_svhn = deepcopy(config)

config.data = configs.DataConfig(
    dataset="mydataset",
    data_dir=config.data.data_dir.parent / "mydataset",
    pin_memory=True,
    img_size=(32, 32, 1),
    num_workers=12,
    num_classes=10,
    reproduce_confidnet_splits=True,
    augmentations={
        "train": {
            "to_tensor": None,
            "random_crop": [32, 4],
            "normalize": [[0.5], [0.5]],
        },
        "val": {
            "to_tensor": None,
            "resize": 32,
            "normalize": [[0.5], [0.5]],
        },
        "test": {
            "to_tensor": None,
            "resize": 32,
            "normalize": [[0.5], [0.5]],
        },
    },
)
config.eval.query_studies.iid_study = "mydataset"
config.eval.query_studies.noise_study = []
config.eval.query_studies.in_class_study = ["svhn"]
config.eval.query_studies.new_class_study = ["cifar10"]
print(config)

Let's take a look at the data loader this configuration will create.

In [ ]:
datamodule = AbstractDataLoader(config)

datamodule.setup()
datamodule.prepare_data()

x, y = next(iter(datamodule.train_dataloader()))


def tensor_to_image(t: torch.Tensor):
    return t.cpu().numpy().transpose(1, 2, 0)


fig, ax = plt.subplots()
ax.imshow(tensor_to_image(x[0]))
fig.show()

We can now train and test on this dataset and recompute our results table. It will now display both SVHN and our new dataset.

In [ ]:
group_name = config.data.dataset
name = "my_first_experiment"
group_dir = config.exp.group_dir.parent / group_name
exp_dir = group_dir / name
exp_dir.mkdir(exist_ok=True, parents=True)
version = exp_utils.get_next_version(exp_dir)

config.exp = configs.ExperimentConfig(
    group_name=group_name,
    name=name,
    mode=configs.Mode.train,
    fold=0,
    crossval_n_folds=0,
    global_seed=1234,
    version=version,
    work_dir=os.getcwd(),
    data_root_dir=os.getenv("DATASET_ROOT_DIR"),
    group_dir=group_dir,
    dir=exp_dir,
    version_dir=exp_dir / f"version_{version}",
    output_paths=configs.OutputPathsPerMode(
        test=configs.OutputPathsConfig(
            raw_output=exp_dir / "test_results" / "raw_logits.npz",
            raw_output_dist=exp_dir / "test_results" / "raw_logits_dist.npz",
            external_confids=exp_dir / "test_results" / "external_confids.npz",
            external_confids_dist=exp_dir
            / "test_results"
            / "external_confids_dist.npz",
        )
    ),
)

print(config)

train(config, RichProgressBar())

config.test = configs.TestConfig(
    name="test_results",
    dir=exp_dir / "test_results",
    cf_path=exp_dir / "hydra/config.yaml",
    selection_criterion="latest",
    best_ckpt_path=exp_dir / f"version_{version}/latest.ckpt",
    only_latest_version=True,
    devries_repro_ood_split=False,
    assim_ood_norm_flag=False,
    iid_set_split="devries",
    raw_output_path="raw_logits.npz",
    external_confids_output_path="external_confids.npz",
    selection_mode="max",
    output_precision=64,
)


print(config)

test(config)

data = pd.concat(
    [
        reporting.load_file(p, experiment_override=str(p.parent.parent.parent.stem))
        for p in Path(os.getenv("EXPERIMENT_ROOT_DIR")).glob("**/test_results/*.csv")
    ]
)
# data = data.assign(experiment=config.exp.group_name)
data = data.assign(study=data.experiment + "_" + data.study)
data = reporting.assign_hparams_from_names(data)

# data = reporting.filter_unused(data)
data = reporting.rename_confids(data)
data = reporting.rename_studies(data)
data = reporting.tables.aggregate_over_runs(data)
data = reporting.str_format_metrics(data)

results_table = reporting.tables.build_results_table(
    data=data, metric="aurc", original_mode=False, paper_filter=False
)
results_table

## Adding a New CSF

We can also add a new softmax-based confidence scoring function, we just have to tell FD-Shifts about it. Afterwards we need to rerun the analysis for both of our experiments and update the results table.

In [ ]:
@analysis.confid_scores.register_confid_func("my_csf")
def my_fancy_csf(softmax):
    return 1 - np.min(softmax, axis=1)


config.eval.confidence_measures.test.append("my_csf")

analysis.main(
    in_path=config.test.dir,
    out_path=config.test.dir,
    query_studies=config.eval.query_studies,
    add_val_tuning=config.eval.val_tuning,
    threshold_plot_confid=None,
    cf=config,
)

config_svhn.eval.confidence_measures.test.append("my_csf")

analysis.main(
    in_path=config_svhn.test.dir,
    out_path=config_svhn.test.dir,
    query_studies=config_svhn.eval.query_studies,
    add_val_tuning=config_svhn.eval.val_tuning,
    threshold_plot_confid=None,
    cf=config_svhn,
)

In [ ]:
data = pd.concat(
    [
        reporting.load_file(p, experiment_override=str(p.parent.parent.parent.stem))
        for p in Path(os.getenv("EXPERIMENT_ROOT_DIR")).glob("**/test_results/*.csv")
    ]
)
# data = data.assign(experiment=config.exp.group_name)
data = data.assign(study=data.experiment + "_" + data.study)
data = reporting.assign_hparams_from_names(data)

# data = reporting.filter_unused(data)
data = reporting.rename_confids(data)
data = reporting.rename_studies(data)
data = reporting.tables.aggregate_over_runs(data)
data = reporting.str_format_metrics(data)

results_table = reporting.tables.build_results_table(
    data=data, metric="aurc", original_mode=False, paper_filter=False
)
results_table